In [1]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

In [2]:
import numpy as np
import pandas as pd
from scipy import optimize, special

F = np.array([[1.0000, 0.1000],
              [0, 1.0000]])

G = np.array([[0.0050],
              [0.1000]])

C = np.array([[1, 0]])

D = 0
Gain = np.array([16.0302, 5.6622])

L = np.array([[0.9902],
              [0.9892]])
safex = [26,31]

In [3]:
import numpy as np
import pandas as pd
from scipy import optimize, special

F = np.array([[1.0000, 0.1000],
              [0, 1.0000]])

G = np.array([[0.0050],
              [0.1000]])

C = np.array([[1, 0]])

D = 0
Gain = np.array([16.0302, 5.6622])

L = np.array([[0.9902],
              [0.9892]])
safex = [26,31]

In [4]:
k=14
inputs = np.random.normal(0, 1, k)
input_len=len(inputs)
X = np.zeros((2, input_len + 1))
Xe = np.zeros((2, input_len + 1))
e = np.zeros((2, input_len))
U = np.zeros((1,input_len))
Y = np.zeros((1,input_len))
r = np.zeros((1,input_len))
z = np.zeros((1,input_len))
S = np.zeros((input_len))
n1 = np.random.normal(0,0.01, [1, input_len])     
n2 = np.random.normal(0,0.001, [2, input_len])

In [5]:
for i in range(input_len):
    e[:, i] = X[:, i] - Xe[:, i]
    U[:,i] = -(Gain @ Xe[:, i])
    X[:, i+1] = F @ X[:, i] + G @ U[:, i] + n2[:, i]
    Y[:,i] = C @ X[:, i] + n1[:,i]
    r[:,i] = Y[:,i] - C @ Xe[:, i]
    Xe[:, i+1] = F @ Xe[:, i] + G @ U[:,i] + L @ r[:,i]
covn_r=np.cov(r)

In [6]:
attackK= [0, 0.04155838 , 0.63838859 , 1.75426837 ,2.44935936 , 3.6651983 , 5.16561739 , 6.95368618 , 9.11716362 ,11.67267658 ,14.29706254, 17.41085821  ,0.       ,   0.    ]
k=len(attackK)
attack2K = [-30.971675   ,-30.971675  ,  -1.11684483 ,-30.90514398, -28.29714574 , -28.61888554, -28.91105434, -29.17567371, -29.41469648, -29.21724115 , -28.49741602 , 30.971675,     0.     ,      0. ]
k1=len(attack2K)

In [7]:
inputs = np.random.normal(0, 1, k)
input_len=len(inputs)
Xk = np.zeros((2, input_len + 1))
Xek = np.zeros((2, input_len + 1))
ek = np.zeros((2, input_len))
Uk = np.zeros((1,input_len))
Uak = np.zeros((1,input_len))
Yk = np.zeros((1,input_len))
rk = np.zeros((1,input_len))
covn_eK = np.zeros((2, 2))
covn_rK = np.zeros((2, 2))
zk = np.zeros((1,input_len))
Sk = np.zeros((input_len))
n1 = np.random.normal(0,0.01, [1, input_len])      
n2 = np.random.normal(0,0.001, [2,input_len])

In [8]:
residuek=[]
mzk=[]
bias=4
for i in range(input_len):
    ek[:, i] = Xk[:, i] - Xek[:, i]
    Uk[:,i] = -(Gain @ Xek[:, i])
    Uak[:,i] = -(Gain @ Xek[:, i])+attack2K[i]
    Xk[:, i+1] = F @ Xk[:, i] + G @ Uak[:, i] + n2[:, i]
    Yk[:,i] = C @ Xk[:, i] + n1[:,i] + attackK[i]
    rk[:,i] = Yk[:,i] - C @ Xek[:, i]
    Xek[:, i+1] = F @ Xek[:, i] + G @ Uk[:,i] + L @ rk[:,i]
    zk[:,i] = rk[:,i].T * covn_r * rk[:,i]
    residuek.append(rk[:,i])
    mzk.append(zk[:,i])
    Sk[i] = Sk[i-1] + zk[:,i] - bias

In [9]:
m=np.mean(np.array(mzk))

In [10]:
r_mean=np.array([])
for i in range(len(residuek)):
    sums=0
    for j in range(i+1):
        sums+=residuek[j]
    mean=sums/(j+1)
    r_mean=np.append(r_mean,mean)

In [11]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [12]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action,next_state, reward, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action,next_state, reward, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward,  done = map(np.stack, zip(*batch))
        return state, action, reward,  done
    
    def __len__(self):
        return len(self.buffer)

In [13]:
class NormalizedActions(gym.ActionWrapper):

    def _action(self, action):
        low_bound   = self.action_space.low
        upper_bound = self.action_space.high
        
        action = low_bound + (action + 1.0) * 0.5 * (upper_bound - low_bound)
        action = np.clip(action, low_bound, upper_bound)
        
        return action

    def _reverse_action(self, action):
        low_bound   = self.action_space.low
        upper_bound = self.action_space.high
        
        action = 2 * (action - low_bound) / (upper_bound - low_bound) - 1
        action = np.clip(action, low_bound, upper_bound)
        
        return actions

In [14]:
class OUNoise(object):
    def __init__(self, action_space, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.3, decay_period=100000):
        self.mu           = mu
        self.theta        = theta
        self.sigma        = max_sigma
        self.max_sigma    = max_sigma
        self.min_sigma    = min_sigma
        self.decay_period = decay_period
        self.action_dim   = action_space.shape[0]
        self.low          = action_space.low
        self.high         = action_space.high
        self.reset()
        
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
        
    def evolve_state(self):
        x  = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)
        self.state = x + dx
        return self.state
    
    def get_action(self, action, t=0):
        ou_state = self.evolve_state()
        self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
        return np.clip(action + ou_state, self.low, self.high)
    
#https://github.com/vitchyr/rlkit/blob/master/rlkit/exploration_strategies/ou_strategy.py

In [15]:
# def plot(frame_idx, rewards):
#     clear_output(True)
#     plt.figure(figsize=(20,5))
#     plt.subplot(131)
#     plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
#     plt.plot(rewards)
#     plt.show()

In [16]:
# class Optimization():
    
#     def __init__(self,params,m,N,Ek):
#         self.params=params.T
#         self.threshold=params[:,0]
#         self.bias=params[:,1]
#         self.m=m
#         self.N=N
#         self.Ek=Ek

#     def gradient(self):
#         x=np.zeros((len(self.Ek)))
#         z=np.zeros((len(self.Ek)))
#         t=np.zeros((len(self.Ek)))
#         dt_dth=np.zeros((len(self.Ek)))
#         dt_dbias=np.zeros((len(self.Ek)))
#         for i in range(len(self.Ek)):
#             x[i]=self.N-1-self.Ek[i]
#             delSi = 2 * self.threshold[i] / (2 * self.N - 1)
#             z[i] = np.maximum(x[i] * delSi + 0.5 * delSi + self.bias[i], 0)
#             t[i] = 1 - special.gammainc(m / 2, z[i]/ 2)
#             dt_dth[i] = 0.5 * z[i] * np.exp(-z[i] / 2) * x[i] / (2 *self.N - 1)
#             dt_dbias[i] = 0.5 * z[i] * np.exp(-z[i] / 2)
#         return np.array([-dt_dth, -dt_dbias])

#     def adam_optimizer(self,learning_rate=0.8, beta1=0.9, beta2=0.999, epsilon=1e-8, num_iterations=2):
#         p = np.zeros_like(self.params)
#         v = np.zeros_like(self.params)
#         initial_values=self.params

#         for t in range(1,num_iterations+1):
#             gradient = self.gradient()
#             p = beta1 * p + (1 - beta1) * gradient
#             v = beta2 * v + (1 - beta2) * (gradient ** 2)
#             p_hat = p / (1 - beta1 ** t)
#             v_hat = v / (1 - beta2 ** t)
#             initial_values += learning_rate * p_hat / (np.sqrt(v_hat) + epsilon)

#         return initial_values

In [17]:
import torch.nn.functional as F
class ValueNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3):
        super(ValueNetwork, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, 1)
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
        x = torch.cat([state, action],1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
    

class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3):
        super(PolicyNetwork, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, num_actions)
        
        self.linear3.weight.data.uniform_(5, 6)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = F.tanh(self.linear3(x))
        return x
    
    def get_action(self, state):
        state  = torch.FloatTensor(state).unsqueeze(0).to(device)
        action = self.forward(state)
        return action.detach().cpu().numpy()[0, 0],action.detach().cpu().numpy()[0, 0]

In [19]:
from scipy.special import erf
import scipy.special as special
from scipy import special, optimize
def ddpg_update(batch_size,Ek,m,
           gamma = 0.99,
           min_value=-np.inf,
           max_value=np.inf,
           soft_tau=1e-2):
    
    state, action,next_state, reward, done = replay_buffer.sample(batch_size)
    state      = torch.FloatTensor(state).to(device)
    next_state = torch.FloatTensor(next_state).to(device)
    action     = torch.FloatTensor(action).to(device)
    reward     = torch.FloatTensor(reward).unsqueeze(1).to(device)
    done       = torch.FloatTensor(np.float32(done)).unsqueeze(1).to(device)

    policy_loss = value_net(state, policy_net(state))
    policy_loss = -policy_loss.mean()
            
    x=N-1-Ek

    next_action    = target_policy_net(next_state)
    th=next_action[0]
    bias=next_action[1]
    delSi = 2 * th / (2 * N - 1)
    z = np.maximum(x * delSi + 0.5 * delSi + bias, 0)
    target_value   = target_value_net(next_state, next_action.detach())
    expected_value = reward + 1 - special.gammainc(m / 2, z/ 2)
    expected_value = torch.clamp(expected_value, min_value, max_value)

    value = value_net(state, action)
    value_loss = value_criterion(value, expected_value.detach())


    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

In [20]:
import numpy as np

class RLEnvironment():
    def __init__(self, num_states,num_action,cusum,residual,mzk,N):
        self.num_states = num_states
        self.num_actions = num_action
        self.cusum = cusum
        self.residual = residual
        self.absorb=N
        self.mzk=mzk
        self.current_state = None
        
    def discrete_frame(self,threshold):
        delSi = 2*threshold/(2*self.absorb-1)
        Ek=np.zeros((len(self.cusum)))
        for i in range(len(self.cusum)):
            if self.cusum[i]<delSi[i]/2:
                Ek[i]=0
            elif self.cusum[i]>=threshold[i]:
                Ek[i]=self.absorb
            else :
                Ek[i]=math.ceil(math.floor(self.cusum[i]/(delSi[i]/2))/2)
        return Ek.reshape(1,-1)
        
    def reset(self,bias):
        self.cusum = np.zeros((input_len))
        for i in range(len(self.cusum)):
            if i>0:
                self.cusum[i]=self.cusum[i-1]+self.mzk[i]-bias[i]
            else:
                self.cusum[i]=self.mzk[i]-bias[i]
        return self.cusum
    
    def calculate_reward(self,action):
        # Calculate reward based on attack detection condition
        if self.residual[i]>0.01 and self.cusum[i] > action[0]:
            return 10
        elif self.residual[i]<0.01 and self.cusum[i] < action[0]:
            return 1
        return 0

    def step(self,action,i):
        b=action[1]
        
        next_state = self.cusum[i-1]+self.mzk[i]-b
        
        reward = self.calculate_reward(action)
        
        self.current_state = next_state
        done = all(self.current_state == self.absorb)
        
        return  next_state, reward, done


In [21]:
# env = NormalizedActions(gym.make("Pendulum-v0"))
# ou_noise = OUNoise(env.action_space)
num_action=2
num_states=2
N=10
env = RLEnvironment(num_states,num_action,Sk,r_mean,mzk,N)
state_dim  = env.num_states
action_dim = env.num_actions
hidden_dim = 256

value_net  = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

target_value_net  = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
target_policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)
    
    
value_lr  = 1e-3
policy_lr = 1e-4

value_optimizer  = optim.Adam(value_net.parameters(),  lr=value_lr)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)

value_criterion = nn.MSELoss()

replay_buffer_size = 1000000
replay_buffer = ReplayBuffer(replay_buffer_size)

In [22]:
max_frames  = 12
# max_steps   = 5
frame_idx   = 0
rewards     = []
batch_size  = 128
th=np.ones((len(Sk),1))*6
bias=np.ones((len(Sk),1))*4

In [23]:
while frame_idx < max_frames:
    state = env.reset(bias)
    max_steps=len(state)
#     ou_noise.reset()
    episode_reward = 0
    E=env.discrete_frame(th)
    for step in range(1,max_steps):
        th[step],bias[step] = policy_net.get_action(np.array([state[step],state[step]]))
#         action = ou_noise.get_action(action, step)
        action=[th[step],bias[step]]
        next_state,reward, done = env.step(action,step)
        replay_buffer.push(state[step], action,next_state, reward, done)
        if len(replay_buffer) > batch_size:
            ddpg_update(batch_size,E[step],m)
        episode_reward += reward
        frame_idx += 1
        
#         if frame_idx % max(1000, max_steps + 1) == 0:
#             plot(frame_idx, rewards)
        
        if done:
            break
    
    rewards.append(episode_reward)

In [24]:
step=1
policy_net.get_action(np.array([state[step],state[step]]))

(1.0, 1.0)

In [25]:
action

[array([1.]), array([1.])]

In [26]:
rewards

[13]

In [27]:
bias

array([[4.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])